In [2]:
import sys
sys.path.append("../..") 

from custom_scripts import database
from custom_scripts import preprocessing

In [3]:
### Table of unique carrier identifiers present in the test samples, and a count of their frequency
carriers = database.query(""" 
                            SELECT op_unique_carrier, COUNT(*)
                                FROM flights_test
                                WHERE fl_date = ANY('{2020-01-01, 2020-01-02, 2020-01-03, 2020-01-04, 2020-01-05, 2020-01-06, 2020-01-07}')
                                GROUP BY op_unique_carrier;
                            """)
carriers.to_csv('../../data/preprocessing/unique_carriers.csv', index=False)
carriers.head(1)

,op_unique_carrier,count
0,9E,4942


In [4]:
### Table of flight numbers present in the test samples, and a count of their frequency
test_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*) 
                                    FROM flights_test
                                    WHERE fl_date = ANY('{{2020-01-31, 2020-01-30, 2020-01-29, 2020-01-28, 2020-01-27, 2020-01-26, 2020-01-25}}')
                                    GROUP BY op_carrier_fl_num;
                                """)
test_flight_numbers.to_csv('../../data/preprocessing/test_flight_numbers.csv', index=False)
test_flight_numbers.head(1)

,op_carrier_fl_num,count
0,6114,14


In [2]:
### Table of historic averages grouped by flight number
averages_by_flight = preprocessing.create_average_table('op_carrier_fl_num','fl_num')
averages_by_flight.to_csv('../../data/preprocessing/averages_by_fl_num.csv', index=False)
averages_by_flight.head(1)

,op_carrier_fl_num,fl_num_avg_dep_delay,fl_num_avg_taxi_out,fl_num_avg_wheels_off,fl_num_avg_wheels_on,fl_num_avg_taxi_in,fl_num_avg_arr_delay,fl_num_avg_crs_elapsed_time,fl_num_avg_actual_elapsed_time,fl_num_avg_air_time,fl_num_avg_carrier_delay,fl_num_avg_weather_delay,fl_num_avg_nas_delay,fl_num_avg_security_delay,fl_num_avg_late_aircraft_delay,fl_num_avg_total_add_gtime,fl_num_avg_longest_add_gtime
0,1,2.7734,16.922582,858.495249,1063.479598,8.920347,-5.716606,262.095996,256.171661,230.33492,24.254237,1.391949,12.355932,0.254237,6.618644,22.192308,22.192308


In [3]:
### Table of historic averages grouped by tail number
averages_by_flight = preprocessing.create_average_table('tail_num','tail_num')
averages_by_flight.to_csv('../../data/preprocessing/averages_by_tail_num.csv', index=False)
averages_by_flight.head(1)

,tail_num,tail_num_avg_dep_delay,tail_num_avg_taxi_out,tail_num_avg_wheels_off,tail_num_avg_wheels_on,tail_num_avg_taxi_in,tail_num_avg_arr_delay,tail_num_avg_crs_elapsed_time,tail_num_avg_actual_elapsed_time,tail_num_avg_air_time,tail_num_avg_carrier_delay,tail_num_avg_weather_delay,tail_num_avg_nas_delay,tail_num_avg_security_delay,tail_num_avg_late_aircraft_delay,tail_num_avg_total_add_gtime,tail_num_avg_longest_add_gtime
0,215NV,10.989011,13.031648,1337.421978,1521.715354,6.647602,8.087588,145.290294,142.429577,122.756602,23.925197,3.667323,9.411417,0.114173,26.240157,29.944444,29.944444


In [4]:
### Table of historic averages grouped by origin
averages_by_flight = preprocessing.create_average_table('origin','origin')
averages_by_flight.to_csv('../../data/preprocessing/averages_by_origin.csv', index=False)
averages_by_flight.head(1)

,origin,origin_avg_dep_delay,origin_avg_taxi_out,origin_avg_wheels_off,origin_avg_wheels_on,origin_avg_taxi_in,origin_avg_arr_delay,origin_avg_crs_elapsed_time,origin_avg_actual_elapsed_time,origin_avg_air_time,origin_avg_carrier_delay,origin_avg_weather_delay,origin_avg_nas_delay,origin_avg_security_delay,origin_avg_late_aircraft_delay,origin_avg_total_add_gtime,origin_avg_longest_add_gtime
0,ABE,10.58383,15.252279,1233.428985,1350.64044,10.775808,7.024949,117.982622,114.84877,88.828178,24.29115,4.138496,17.511504,0.044248,33.039823,48.3,47.442857


In [5]:
### Table of historic averages grouped by dest
averages_by_flight = preprocessing.create_average_table('dest','dest')
averages_by_flight.to_csv('../../data/preprocessing/averages_by_dest.csv', index=False)
averages_by_flight.head(1)

,dest,dest_avg_dep_delay,dest_avg_taxi_out,dest_avg_wheels_off,dest_avg_wheels_on,dest_avg_taxi_in,dest_avg_arr_delay,dest_avg_crs_elapsed_time,dest_avg_actual_elapsed_time,dest_avg_air_time,dest_avg_carrier_delay,dest_avg_weather_delay,dest_avg_nas_delay,dest_avg_security_delay,dest_avg_late_aircraft_delay,dest_avg_total_add_gtime,dest_avg_longest_add_gtime
0,ABE,9.510175,20.078658,1507.55495,1613.605655,4.902223,4.592501,106.034767,101.577833,76.608659,23.426673,5.866636,10.776352,0.095325,25.921632,41.986842,41.197368


In [6]:
### Table of historic averages grouped by op_unique_carrier
averages_by_flight = preprocessing.create_average_table('op_unique_carrier','carrier')
averages_by_flight.to_csv('../../data/preprocessing/averages_by_carrier.csv', index=False)
averages_by_flight.head(1)

,op_unique_carrier,carrier_avg_dep_delay,carrier_avg_taxi_out,carrier_avg_wheels_off,carrier_avg_wheels_on,carrier_avg_taxi_in,carrier_avg_arr_delay,carrier_avg_crs_elapsed_time,carrier_avg_actual_elapsed_time,carrier_avg_air_time,carrier_avg_carrier_delay,carrier_avg_weather_delay,carrier_avg_nas_delay,carrier_avg_security_delay,carrier_avg_late_aircraft_delay,carrier_avg_total_add_gtime,carrier_avg_longest_add_gtime
0,9E,10.550252,21.399104,1360.586632,1463.700776,7.399465,3.788254,108.949979,102.194651,73.410588,20.54492,5.287241,21.227495,0.019339,31.674297,47.894401,46.809968
